# First Step is to preprocess a .gz file to isolate one pods worth of data

In [68]:
import gzip
import pandas as pd
import json
pd.set_option('display.max_columns', None)
import numpy as np

In [8]:
def gz2df(path):
    with gzip.open(path, 'rb') as f:
        """
        Convert .gz log files into dataframe
        """
        content = f.read().decode('utf-8').split('\n')
        list_rows = []

        for i in range(len(content)-1):
            row = content[i].split(' ')
            list_rows.append(row)
        f.close()

    df = pd.DataFrame(list_rows, columns=['log_timestamp', 'data'])

    return df

def get_type(x):
    idx1 = x.find('"Type":') + 8
    idx2 = x.find(',', idx1) - 1
    rec_type = x[ idx1:idx2 ]

    return rec_type

def filter_by_rec_type(df, rec_type): 
    return df[df.rec_type == rec_type]

def explode_df(df):
    return pd.concat([df.log_timestamp, pd.json_normalize(df.data)], axis=1)


In [21]:
streamed_data_file = "./node1.gz"

df = gz2df(streamed_data_file)
df['rec_type'] = df.data.apply(get_type)
df['data'] = df.data.apply(json.loads)



list_of_df_rectypes = {}  

for rec_type in df.rec_type.unique():
    print(rec_type)
    rec_type_df = filter_by_rec_type(df, rec_type)
    df_exploded = explode_df(rec_type_df)
    list_of_df_rectypes[rec_type]=df_exploded
    


PodNet
NodeNet
NodeFS
Container
Pod
NodeDiskIO
Node


In [36]:
df_pod = list_of_df_rectypes["Pod"]

In [48]:
print(df_pod.PodName.unique())

['kube-proxy' 'open5gs-upf' 'open5gs-mongodb' 'cloudwatch-agent'
 'fluent-bit' 'open5gs-webui' 'aws-node' 'open5gs-udm' 'ebs-csi-node'
 'ueransim-ues-first-batch' 'open5gs-nrf' nan]


In [74]:
upf_df = df_pod[df_pod["PodName"]=="open5gs-upf" ]

In [77]:
## cleaning up the nans in the dataframe

print(upf_df['pod_cpu_usage_total'].isna().sum())
print(len(upf_df))

upf_df = upf_df.dropna(subset=['pod_cpu_usage_total', 'pod_memory_max_usage' ])

23
240


# Model Time

In [82]:

limit_cpu = max( upf_df[["pod_cpu_usage_total"]].values.tolist() )


request_cpu = np.mean( upf_df[["pod_cpu_usage_total"]].values.tolist() ) 

limit_memory =   max( upf_df[["pod_memory_max_usage"]].values.tolist() )
request_memory = np.mean( upf_df[["pod_memory_max_usage"]].values.tolist() ) 

# Time to postprocess